In [1]:
from neo4j import GraphDatabase
import os
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"


In [3]:
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "azerty92")

with GraphDatabase.driver(URI, auth=AUTH) as driver: 
    driver.verify_connectivity()

In [5]:
def count_row(tx, fn):  
    result = tx.run(
        f"""
        LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
        RETURN COUNT(row)
        """ 
    )
    count = result.fetch(1)
    summary = result.consume()
    return count, summary

def exec_write(query, csv_filename, db):
    # Execute a write query in the provided DB from a csv
    with driver.session(database=db) as session:
        summary = session.run(query.format(fn=csv_filename)).consume()

        print("Created {nodes_created} nodes and {relationship_created} relationships and set {properties_set} properties in {time} ms.".format(
            nodes_created=summary.counters.nodes_created,
            properties_set=summary.counters.properties_set,
            relationship_created=summary.counters.relationships_created,
            time=summary.result_available_after
        ))

In [8]:
create_country = """
LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row 
WITH row WHERE row.FIPS IS NOT NULL 
MERGE (c:Country {{FIPS: row.FIPS, name: row.Name}});  
"""
exec_write(create_country, "fips.csv", "gdelt")

Created 279 nodes and 0 relationships and set 558 properties in 119 ms.


# Create Events and actors

In [17]:
# Count lines in csv
with driver.session(database="gdelt") as session:
    count, summary = session.execute_write(count_row, fn='batch_2022-01-01_00:00:00_2022-01-14_23:45:00_export.csv')  

    print("Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after
    ))
    print(count)

Created 0 nodes in 1059 ms.
[<Record COUNT(row)=332>]


In [6]:
# Create event nodes
create_event = """
LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
WITH row WHERE row.GlobalEventID IS NOT NULL
CALL {{
    WITH row
    MERGE (event:Event {{globalEventID:row.GlobalEventID}})
    ON CREATE
    SET
        event.date = Datetime(row.Day),
        event.type = row.EventCode
}} IN TRANSACTIONS OF 15000 ROWS;
"""

exec_write(create_event, "batch_2022-01-01_00:00:00_2022-01-14_23:45:00_export.csv", "gdelt")

Created 1912585 nodes and 0 relationships and set 5737755 properties in 58446 ms.


In [9]:
# Create relationship between event and country

create_event_country = """
LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
WITH row, COALESCE(row.ActionGeo_Lat, null) AS latitude, COALESCE(row.ActionGeo_Long, null) AS longitude
WHERE row.ActionGeo_CountryCode IS NOT NULL
CALL {{
    WITH row, latitude, longitude
    MATCH (event:Event {{globalEventID : row.GlobalEventID}})
    MATCH (country:Country {{FIPS : row.ActionGeo_CountryCode}})
    CREATE (event)-[:TAKES_PLACE {{lat: toFloat(latitude), lon: toFloat(longitude)}}]->(country)
}} IN TRANSACTIONS OF 15000 ROWS;
"""
exec_write(create_event_country, "batch_2022-01-01_00:00:00_2022-01-14_23:45:00_export.csv", "gdelt")

Created 0 nodes and 1840425 relationships and set 3680710 properties in 171950 ms.


In [10]:
# Create actor1 node

# actor1Name idex: 6
# actor2Name idex: 16
create_actor1 = """ 
            LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
            WITH row WHERE row.Actor1Name IS NOT NULL
            CALL {{
                WITH row
                MERGE (:Actor {{name:row.Actor1Name}})
            }} IN TRANSACTIONS OF 15000 ROWS;
        """
exec_write(create_actor1, "batch_2022-01-01_00:00:00_2022-01-14_23:45:00_export.csv", "gdelt")

Created 5491 nodes and 0 relationships and set 5491 properties in 12722 ms.


In [12]:
# Create actor2 node

# actor1Name idex: 6
# actor2Name idex: 16
create_actor2 = """ 
            LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
            WITH row WHERE row.Actor2Name IS NOT NULL
            CALL {{
                WITH row
                MERGE (:Actor {{name:row.Actor2Name}})
            }} IN TRANSACTIONS OF 15000 ROWS;
        """
exec_write(create_actor2, "batch_2022-01-01_00:00:00_2022-01-14_23:45:00_export.csv", "gdelt")

Created 407 nodes and 0 relationships and set 407 properties in 11062 ms.


In [13]:
# Create relationship between actor1 and event

# actor1Name idex: 6
# actor2Name idex: 16
create_actor1_event = """
            LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
            WITH row WHERE row.Actor1Name IS NOT NULL
            CALL {{
                WITH row
                MATCH (event:Event {{globalEventID : row.GlobalEventID}})
                MATCH (actor:Actor {{name : row.Actor1Name}})
                MERGE (actor)-[:ACTS_IN {{actor_type:1}}]->(event)
            }} IN TRANSACTIONS OF 15000 ROWS;
        """
exec_write(create_actor1_event, "batch_2022-01-01_00:00:00_2022-01-14_23:45:00_export.csv", "gdelt")

Created 0 nodes and 1704299 relationships and set 1704299 properties in 72534 ms.


In [14]:
# Create relationship between event and actor2

# actor1Name idex: 6
# actor2Name idex: 16
create_event_actor2 = """
            LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
            WITH row WHERE row.Actor2Name IS NOT NULL
            CALL {{
                WITH row
                MATCH (event:Event {{globalEventID : row.GlobalEventID}})
                MATCH (actor:Actor {{name : row.Actor2Name}})
                MERGE (actor)-[:ACTS_IN {{actor_type:2}}]->(event)
            }} IN TRANSACTIONS OF 15000 ROWS;
        """
exec_write(create_event_actor2, "batch_2022-01-01_00:00:00_2022-01-14_23:45:00_export.csv", "gdelt")

Created 0 nodes and 1266050 relationships and set 1266050 properties in 55506 ms.


# Create Mentions and Resources

In [6]:
create_resource = """ 
            LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
            WITH row WHERE row.MentionIdentifier IS NOT NULL
            CALL {{
                WITH row
                MERGE (:Resource {{originalID:row.MentionIdentifier}})
            }} IN TRANSACTIONS OF 15000 ROWS;
        """
exec_write(create_resource, "batch_2022-01-01_00:00:00_2022-02-01_00:00:00_mentions.csv", "gdelt")

Created 2579007 nodes and 0 relationships and set 2579007 properties in 151241 ms.


In [7]:
create_resource_event = """ 
            LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
            WITH row WHERE row.MentionIdentifier IS NOT NULL AND row.GlobalEventID IS NOT NULL
            CALL {{
                WITH row
                MATCH (event:Event {{globalEventID : row.GlobalEventID}})
                MATCH (resource:Resource {{originalID : row.MentionIdentifier}})
                CREATE (resource)-[:MENTIONS {{date:Datetime(substring(row.EventTimeDate, 0, 8)), confidence:coalesce(toInteger(row.Confidence), 0)}}]->(event)
            }} IN TRANSACTIONS OF 15000 ROWS;
        """
exec_write(create_resource_event, "batch_2022-01-01_00:00:00_2022-02-01_00:00:00_mentions.csv", "gdelt")

Created 0 nodes and 11286181 relationships and set 22572362 properties in 529587 ms.


# GKG

In [10]:
create_themes = """ 
            LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
            WITH row WHERE row.Themes IS NOT NULL
            CALL {{
                WITH row
                UNWIND split(row.Themes, ';') as theme
                MERGE (:Theme {{name:theme}})
            }} IN TRANSACTIONS OF 15000 ROWS;
        """
exec_write(create_themes, "batch_2022-01-01_000000_2022-01-05_234500_gkg.csv", "gdelt")

Created 14858 nodes and 0 relationships and set 14858 properties in 117858 ms.


In [11]:
create_sources = """ 
            LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
            WITH row WHERE row.SourceCommonName IS NOT NULL
            CALL {{
                WITH row
                MERGE (:Source {{name:row.SourceCommonName, type:row.SourceCollectionIdentifier}})
            }} IN TRANSACTIONS OF 15000 ROWS;
        """
exec_write(create_sources, "batch_2022-01-01_000000_2022-01-05_234500_gkg.csv", "gdelt")

Created 16284 nodes and 0 relationships and set 32568 properties in 16618 ms.


In [12]:
update_resource = """ 
            LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
            WITH row WHERE row.DocumentIdentifier IS NOT NULL
            CALL {{
                WITH row
                MATCH (r:Resource {{originalID:row.DocumentIdentifier}})
                SET r += {{date: Datetime(substring(row.DATE, 0, 8)), originalLanguage:coalesce(row.TranslationInfo, "eng"), tone:coalesce(toFloat(split(row.V2Tone, ',')[0]), 0.0)}}
            }} IN TRANSACTIONS OF 15000 ROWS;
        """
exec_write(update_resource, "batch_2022-01-01_000000_2022-01-05_234500_gkg.csv", "gdelt")

Created 0 nodes and 0 relationships and set 1056903 properties in 41496 ms.


In [13]:
create_resource_themes = """ 
            LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
            WITH row WHERE row.Themes IS NOT NULL AND row.DocumentIdentifier IS NOT NULL
            CALL {{
                WITH row
                UNWIND split(row.Themes, ';') as theme
                MATCH (t:Theme {{name:theme}})
                MATCH (resource:Resource {{originalID : row.DocumentIdentifier}})
                CREATE (resource)-[:HAS]->(t)
            }} IN TRANSACTIONS OF 15000 ROWS;
        """
exec_write(create_resource_themes, "batch_2022-01-01_000000_2022-01-05_234500_gkg.csv", "gdelt")

Created 0 nodes and 12063421 relationships and set 0 properties in 706546 ms.


In [14]:
create_source_resource = """ 
            LOAD CSV WITH HEADERS FROM 'file:///{fn}' AS row
            WITH row WHERE row.SourceCommonName IS NOT NULL
            CALL {{
                WITH row
                MATCH (source:Source {{name:row.SourceCommonName}})
                MATCH (resource:Resource {{originalID : row.DocumentIdentifier}})
                CREATE (source)-[:PUBLISH]->(resource)
            }} IN TRANSACTIONS OF 15000 ROWS;
        """
exec_write(create_source_resource, "batch_2022-01-01_000000_2022-01-05_234500_gkg.csv", "gdelt")

Created 0 nodes and 352290 relationships and set 0 properties in 53265 ms.


# Requêtes projet

In [ ]:
# afficher le nombre d’articles/évènements qu’il y a eu pour chaque triplet (jour, pays de l’évènement, langue de l’article).
q1 = """
MATCH (r:Resource)-[m:MENTIONS]->(e:Event)-[:TAKES_PLACE]->(c:Country)
RETURN e.date AS jour,
    c.name AS pays,
    r.originalLanguage AS langue_article,
    COUNT(DISTINCT(r)) AS nb_ressources,
    COUNT(DISTINCT(e)) AS nb_event
"""

# pour un pays donné en paramètre, affichez les évènements qui y ont eu place triées par le nombre de mentions (tri décroissant);
# permettez une agrégation par jour/mois/année

q2 = """
MATCH p = (russia:Country {{name:{country}}})<-[:TAKES_PLACE]-(e:Event)<-[m:MENTIONS]-()
RETURN 	m.date.year AS Year, m.date.month AS Month, m.date.day AS Day, e.globalEventID AS Event, COUNT(m) AS nb_mention
ORDER BY nb_mention DESC
"""

# pour une source de donnés passée en paramètre (gkg.SourceCommonName)
# affichez les thèmes, personnes, lieux dont les articles de cette sources parlent
# ainsi que le nombre d’articles et le ton moyen des articles (pour chaque thème/personne/lieu);
# permettez une agrégation par jour/mois/année.

q3 = """
MATCH (s:Source {{name:{source_name}}})-[:PUBLISH]->(r:Resource)-[:MENTIONS]->(e:Event)<-[:ACTS_IN]-(a),
(e)-[:TAKES_PLACE]->(c:Country),
(r)-[:HAS]->(t)
RETURN r.date.year AS Year, 
        r.date.month AS Month, 
        r.date.day AS Day,
        t.name AS Theme, 
        a.name AS Personne, 
        c.name AS Pays, 
        COUNT(r) AS nb_article, 
        AVG(r.tone) AS ton_moyen
ORDER BY ton_moyen DESC
"""

q3_viz = """
MATCH (s:Source {{name:{source_name}}})-[:PUBLISH]->(r:Resource)-[:MENTIONS]->(e:Event)<-[:ACTS_IN]-(),
(e)-[:TAKES_PLACE]->(c:Country),
(r)-[:HAS]->(t)
RETURN *, COUNT(e)
"""

# étudiez l’évolution des relations entre deux pays (specifies en paramètre) au cours de l’année.
# Vous pouvez vous baser sur la langue de l’article, le ton moyen des articles,
# les themes plus souvent citées, les personalités ou tout element qui vous semble pertinent.

q4 = """
MATCH (ukraine:Country {name:"Ukraine"})<-[t:TAKES_PLACE]-(e)<-[:MENTIONS]-(r),
(russia:Country {name:"Russia"})<-[tr:TAKES_PLACE]-(er)<-[:MENTIONS]-(r),
(a)-[:ACTS_IN]->(e),
(ar)-[:ACTS_IN]->(er),
(r)-[:has]->(t:Theme)
RETURN ukraine.name AS country_1,
        russia.name AS country_2,
        r.date.year AS Year, 
        r.date.month AS Month, 
        r.date.day AS Day,
        AVG(r.tone) AS avg_tone
"""